In [ ]:
# If file not downloaded
import zipfile
import pandas as pd
with zipfile.ZipFile("archive.zip", "r") as zip_ref:
    zip_ref.extractall(".")


In [ ]:
# create folder
import os
import time
from datetime import datetime
timenow = datetime.now().strftime('%x,%X')

class extract:
    def __init__(self, input):
        data = input.split(',')
        self.date = data[0].split('/')
        self.time = data[1].split(':')
    def day(self):
        return int(self.date[1])
    def month(self):
        return int(self.date[0])
    def year(self):
        return int(self.date[2])
    def hour(self):
        return int(self.time[0])
    def minute(self):
        return int(self.time[1])
    def second(self):
        return int(self.time[2])
    


    
current = os.getcwd()
folder = "\\theatre"
os.mkdir(current+folder)
if os.path.getsize(current+folder) == 0:
    for filename in os.listdir(current):
        f = os.path.join(current, filename)
#         print(f)
#         print(current+folder)
        modtime = time.strftime('%x,%X', time.localtime(os.path.getmtime(f)))
        filetime = extract(modtime)
        nowtime = extract(timenow)
        if filetime.year() == nowtime.year() and filetime.month() == nowtime.month() and filetime.day() == nowtime.day():
#             print(f,current+folder)
#             print(filetime.day(),nowtime.day())
#             print(filetime.month(),nowtime.month())
#             print(filetime.year(),nowtime.year())
            if f != current+folder:
                os.rename(current+'\\'+filename, current+folder+'\\'+filename)
os.chdir(current+folder)




        
        
        



In [ ]:
os.chdir(current+folder)

In [ ]:
import cv2
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Specify the path to the folder containing the dataset
dataset_folder = os.getcwd()

# Specify the image dimensions
image_width = 224
image_height = 224

# Initialize empty lists to store the data
images = []
ratings = []

# Iterate through each genre folder in the dataset
for genre_folder in os.listdir(dataset_folder):
    genre_path = os.path.join(dataset_folder, genre_folder)
    if os.path.isdir(genre_path):
        # Iterate through each movie image in the genre folder
        for filename in os.listdir(genre_path):
            image_path = os.path.join(genre_path, filename)
            if filename.endswith('.jpg') or filename.endswith('.png'):
                # Load the image using OpenCV
                image = cv2.imread(image_path)
                # Resize the image to the desired dimensions
                image = cv2.resize(image, (image_width, image_height))
                # Convert the image to grayscale
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                # Append the image to the images list
                images.append(image)
                # Extract the movie rating from the metadata
                rating = image.info.get('Rating', None)
                if rating is not None:
                    # Append the movie rating to the ratings list
                    ratings.append(float(rating))

# Convert the lists to NumPy arrays
images = np.array(images)
ratings = np.array(ratings)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, ratings, test_size=0.2, random_state=42)

# Normalize pixel values between 0 and 1
X_train = X_train / 255.0
X_test = X_test / 255.0

# Define the model architecture
model = Sequential()
model.add(Flatten(input_shape=(image_width, image_height)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


In [ ]:
# Save trained model

# Specify the path and filename for saving the model
save_path = current+folder+'\\model.h5'

# Save the model
model.save(save_path)

from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model(save_path)


In [ ]:
# zip up the model
import shutil 
import os.path

# Creating the ZIP file 
archived = shutil.make_archive(save_path, 'zip', save_path)

if os.path.exists(save_path+'.zip'):
   print(archived)
else: 
   print("ZIP file not created")

In [ ]:
# Prediction code
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

# Specify the path to the folder containing the images and metadata
folder_path = 'path/to/folder'

# Load the trained model
model = load_model('path/to/trained_model.h5')

# Initialize empty lists to store the data
images = []
metadata = []

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the image using OpenCV
        image = cv2.imread(file_path)
        # Resize the image to the desired dimensions
        image = cv2.resize(image, (image_width, image_height))
        # Convert the image to grayscale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Append the image to the images list
        images.append(image)
        # Open the image using PIL to extract metadata
        pil_image = Image.open(file_path)
        # Extract the metadata from the image
        title = pil_image.info.get('Title', '')
        subject = pil_image.info.get('Subject', '')
        tags = pil_image.info.get('Tags', '')
        comments = pil_image.info.get('Comments', '')
        # Append the metadata to the metadata list as a single string
        metadata.append(f"{title} {subject} {tags} {comments}")

# Convert the lists to NumPy arrays
images = np.array(images)
metadata = np.array(metadata)

# Normalize pixel values between 0 and 1
images = images / 255.0

# Make predictions using the trained model
predictions = model.predict([images, metadata])

# Print the predicted movie ratings
for i, filename in enumerate(os.listdir(folder_path)):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        print(f"{filename}: {predictions[i][0]}")

